<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# FIA Biomass Modeling

_by Jeremy Freeman (CarbonPlan), September 19, 2020_

This notebook extracts fits predictive biomass growth curves from FIA data

In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import numpy as np
from forests import load, setup, plot, fit

In [3]:
setup.plotting()

In [1349]:
df = load.biomass(store='local', states=['CA'])

In [1350]:
df['type_code'].unique()

array([924., 185., 368., 225., 281., 221., 262., 922., 371., 261., 241.,
       202., 201., 941., 341., 923., 911., 921., 933., 962., 369., 184.,
       974., 934., 935., 367., 931., 361., 270., 226., 365., 942., 363.,
       280., 910., 961., 901., 203., 366., 305., 912., 943., 301., 704.,
       973., 364., 383., 222., 224., 976., 342., 972., 267., 125., 709.])

In [1490]:
inds = df['type_code'] == 261 # 261 is a problem, 941 / 371 is a good example, 341 shows model deviation
x = df[inds]['age']
y = df[inds]['biomass']
plot.xy(x=x, y=y, clim=(500,3500), xlim=[0, 200], ylim=[0, 500])

alt.Chart(...)

In [876]:
def logistic(x, p):
    a, b, c = p
    return a * (1 / (1 + c * np.exp(-b * (x))) - (1 / (1 + c))) * ((c + 1) / c)

In [824]:
x = np.arange(0,200)

In [875]:
print(np.min(logistic(x,[100,0.1,1])))
print(np.max(logistic(x,[100,0.1,5])))

0.0
69.44444349530819


In [861]:
np.exp(-0.2*10000)

0.0

In [1101]:
plot.line(x=x, y=logistic(x,[100,0.1,0.00001]), clim=(500,3500), xlim=[0, 200], ylim=[0, 600])

alt.Chart(...)

In [1500]:
model = fit.growth(x=x, y=y, noise='gamma')
print(model.result)

      fun: 2313.075217206763
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-9.09494702e-05, -2.53658072e-01,  4.54747351e-04, -9.09494702e-05])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 245
      nit: 34
   status: 0
  success: True
        x: array([3.00360176e+02, 1.84645181e-02, 1.36840590e+00, 8.14218054e+01])


In [1501]:
model.r2(x, y)

0.19320663990164422

In [1502]:
yhat = model.predict(x)
samples = model.sample(x)

In [1499]:
xlim = [0, 400]
ylim= [0,600]
((plot.xy(x=x, y=y, xlim=xlim, ylim=ylim)
+ plot.line(x=x, y=yhat, xlim=xlim, ylim=ylim))
|
(plot.xy(x=x, y=samples, xlim=xlim, ylim=ylim)
+ plot.line(x=x, y=yhat, xlim=xlim, ylim=ylim)))

alt.HConcatChart(...)

In [103]:
((plot.xy(x=x, y=y, xlim=[0,200], ylim=[0, 800])
+ plot.line(x=x, y=yhat, xlim=[0,200], ylim=[0,800]))
|
(plot.xy(x=x, y=samples, xlim=[0,200], ylim=[0,800])
+ plot.line(x=x, y=yhat, xlim=[0,200], ylim=[0,800])))

alt.HConcatChart(...)

In [ ]:
plot.carto(lat=df['lat'], lon=df['lon'], color=foo, cmap='blues', clim=(0, 2000))

In [ ]:
import fsspec

In [ ]:
path = setup.loading('local')

In [ ]:
state = 'CA'

In [ ]:
import pandas as pd
import xarray as xr

In [ ]:
df['INVYR'].max()

In [ ]:
df = pd.read_parquet(path / f'processed/fia-states/long/{state.lower()}.parquet')

In [ ]:
path = setup.loading('local')

In [ ]:
mapper = fsspec.get_mapper(path / 'processed/terraclimate/conus/4000m/raster.zarr')

In [ ]:
ds = xr.open_zarr(mapper)

In [ ]:
crs = ds['crs']

In [ ]:
from pyproj import transform, Proj

In [ ]:
ds['ppt']


In [ ]:
Proj?

In [ ]:
from carbonplan_data.utils import albers_conus_crs, albers_conus_extent

In [ ]:
albers_conus_extent()

In [ ]:
from rasterio import Affine

In [ ]:
t = Affine(albers_conus_extent())

In [ ]:
ds

In [ ]:
ds['crs'].to_dict()

In [ ]:
Proj(crs.to_dict())

In [ ]:
ids = df[['LAT', 'LON']].to_xarray()

In [ ]:
ds.sel(lat=ids['LAT'], lon=ids['LON'], method='nearest')